In [ ]:
import matplotlib.pyplot as plt
import matplotlib.transforms as transforms
import numpy as np
import pandas as pd
from matplotlib.patches import Ellipse

In [ ]:
data = pd.read_csv('wine-data_reduced.csv', delimiter=' ', header=None)
data

#### a) Estimate the amount of data points contained in the 1−σ, 2−σ and 3−σ ellipsoid area. Data points contained in these areas exhibit a Mahalanobis distance ≤ 1, ≤ 2 and ≤ 3.

In [ ]:
# Calculate covariance matrix and mean
cov_matrix = np.cov(data.T)
mean = np.mean(data, axis=None)
sigma = np.var(data)
# Calculate Mahalanobis distance for each data point
distances = []
for i in range(len(data)):
    distance = np.sqrt(np.dot(np.dot((data.iloc[i] - mean), np.linalg.inv(cov_matrix)), (data.iloc[i] - mean)))
    distances.append(distance)

# Count number of data points in each region
num_points_1sigma = len([d for d in distances if d <= 1])
num_points_2sigma = len([d for d in distances if d <= 2])
num_points_3sigma = len([d for d in distances if d <= 3])

#### b) Compare your results with those of the traditional 3 − σ rule used for the normal distribution.

In [ ]:

print('Mahalanobis values:')
print('Number of points within 1-sigma ellipsoid: ', num_points_1sigma)
print('Number of points within 2-sigma ellipsoid: ', num_points_2sigma)
print('Number of points within 3-sigma ellipsoid: ', num_points_3sigma)
print('-----------------------------------------------')
print('Normal Distribution:')
print('3-Sigma: ', len(data) * 0.9973)
print('2-Sigma: ', len(data) * 0.9545)
print('1-Sigma: ', len(data) * 0.6827)

In [ ]:
"""
FOUND HERE:
https://matplotlib.org/stable/gallery/statistics/confidence_ellipse.html
"""


def confidence_ellipse(x, y, ax, n_std=3.0, facecolor='none', **kwargs):
    """
    Create a plot of the covariance confidence ellipse of *x* and *y*.

    Parameters
    ----------
    x, y : array-like, shape (n, )
        Input data.

    ax : matplotlib.axes.Axes
        The axes object to draw the ellipse into.

    n_std : float
        The number of standard deviations to determine the ellipse's radiuses.

    **kwargs
        Forwarded to `~matplotlib.patches.Ellipse`

    Returns
    -------
    matplotlib.patches.Ellipse
    """
    if x.size != y.size:
        raise ValueError("x and y must be the same size")

    cov = np.cov(x, y)
    pearson = cov[0, 1] / np.sqrt(cov[0, 0] * cov[1, 1])
    # Using a special case to obtain the eigenvalues of this
    # two-dimensional dataset.
    ell_radius_x = np.sqrt(1 + pearson)
    ell_radius_y = np.sqrt(1 - pearson)
    ellipse = Ellipse((0, 0), width=ell_radius_x * 2, height=ell_radius_y * 2,
                      facecolor=facecolor, **kwargs)

    # Calculating the standard deviation of x from
    # the squareroot of the variance and multiplying
    # with the given number of standard deviations.
    scale_x = np.sqrt(cov[0, 0]) * n_std
    mean_x = np.mean(x)

    # calculating the standard deviation of y ...
    scale_y = np.sqrt(cov[1, 1]) * n_std
    mean_y = np.mean(y)

    transf = transforms.Affine2D() \
        .rotate_deg(45) \
        .scale(scale_x, scale_y) \
        .translate(mean_x, mean_y)

    ellipse.set_transform(transf + ax.transData)
    return ax.add_patch(ellipse)

In [ ]:
def get_correlated_dataset(n, dependency, mu, scale):
    latent = np.random.randn(n, 2)
    dependent = latent.dot(dependency)
    scaled = dependent * scale
    scaled_with_offset = scaled + mu
    # return x and y of the new, correlated dataset
    return scaled_with_offset[:, 0], scaled_with_offset[:, 1]

#### c) Visualize your results.

In [ ]:
fig, ax_nstd = plt.subplots(figsize=(6, 6))

dependency_nstd = [[0.8, 0.75],
                   [-0.2, 0.35]]
mu = 0, 0
scale = 8, 5

ax_nstd.axvline(c='grey', lw=1)
ax_nstd.axhline(c='grey', lw=1)

x, y = get_correlated_dataset(500, dependency_nstd, mu, scale)
ax_nstd.scatter(data.iloc[:, 0], data.iloc[:, 1], s=0.5)

confidence_ellipse(data.iloc[:, 0], data.iloc[:, 1], ax_nstd, n_std=1,
                   label=r'$1\sigma$', edgecolor='firebrick')
confidence_ellipse(data.iloc[:, 0], data.iloc[:, 1], ax_nstd, n_std=2,
                   label=r'$2\sigma$', edgecolor='fuchsia', linestyle='--')
confidence_ellipse(data.iloc[:, 0], data.iloc[:, 1], ax_nstd, n_std=3,
                   label=r'$3\sigma$', edgecolor='blue', linestyle=':')

ax_nstd.scatter(mu[0], mu[1], c='red', s=3)
ax_nstd.set_title('Different standard deviations')
ax_nstd.legend()
plt.show()

#### d) What happens if you normalize the axis by the corresponding eigenvalues?